<a href="https://colab.research.google.com/github/surajsrivathsa/thesis_comics_search_xai/blob/main/feature_extraction/jupyter_notebooks/BookNLP_Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preamble

In [ ]:
!pip install booknlp
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install polyfuzz
!pip install text2emotion
!pip install NRCLex
!pip install autocorrect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 8.0 MB/s 
     |████████████████████████████████| 4.4 MB 58.8 MB/s 
     |████████████████████████████████| 86 kB 2.9 MB/s 
     |████████████████████████████████| 596 kB 11.0 MB/s 
     |████████████████████████████████| 6.6 MB 14.3 MB/s 
  Created wheel for booknlp: filename=booknlp-1.0.7-py3-none-any.whl size=2420766 sha256=86acbf089b857880236ea039e1ef783779b44d6741678fea41981cbd300eb670
  Stored in directory: /root/.cache/pip/wheels/3e/79/30/e49074dd91db7434da782e5d48e44bdabe066c07adf465f376
Successfully built booknlp
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-w

In [ ]:
!git clone https://github.com/jkwieser/personality-detection-text.git

fatal: destination path 'personality-detection-text' already exists and is not an empty directory.


In [ ]:
from booknlp.booknlp import BookNLP
from polyfuzz import PolyFuzz
from polyfuzz.models import TFIDF, EditDistance
from autocorrect import Speller
#Import the modules
import text2emotion as te
from nrclex import NRCLex
import pickle

In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import torch
import pickle
import json
from collections import Counter
import torch
import os, sys, glob, re
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

thesis_filepath = '/content/drive/MyDrive/suraj/thesis'

dataset_path = os.path.join(thesis_filepath, 'dataset', 'emoreccom_public_train')
os.chdir(dataset_path)

Mounted at /content/drive


# preprocess text per book

In [ ]:
raw_text_df = pd.read_csv('combined_text_img_df.csv')
print(raw_text_df.shape)
raw_text_df.head()

(51218, 10)


,comic_no,page_no,panel_no,text,concatenated_text,fastpunct_text,decontracted_text,text_clean,image_id,img_file_path
0,3451,2,0,['mercy but the boy he tries to save is too ho...,mercy but the boy he tries to save is too hot ...,"['Mercy, but the boy he tries to save is too h...","['mercy , but the boy he tries to save is too ...","['mercy , but the boy he tries to save is too ...",3451_2_0,/content/drive/MyDrive/suraj/thesis/raw_panel_...
1,3451,3,0,"['nsion several miles from the grove, althy un...","nsion several miles from the grove , althy un...","['Cinder several miles from the grove, along R...","['cinder several miles from the grove , along ...","['cinder several miles from the grove , along ...",3451_3_0,/content/drive/MyDrive/suraj/thesis/raw_panel_...
2,3451,3,1,"['e e owwwi', 'randy expresses his dislike ey ...",e - e - owwwi. randy expresses his dislike ey ...,"['E e owww.i.', 'Randy expresses his dislike a...","['e e owww .i .', 'randy expresses his dislike...","['e e owww .i .', 'randy expresses his dislike...",3451_3_1,/content/drive/MyDrive/suraj/thesis/raw_panel_...
3,3451,3,2,"['so. take one yourself, meadowsd']","so ... take one yourself , meadowsd","['So take one yourself, meadowsd.']","['so take one yourself , meadowsd .']","['so take one yourself , meadowsd .']",3451_3_2,/content/drive/MyDrive/suraj/thesis/raw_panel_...
4,3451,3,3,"['as meadows flees from the room, randy fills ...","as meadows flees from the room , randy fills ...","['As Meadows flees from the room, Randy fills ...","['as meadows flees from the room , randy fills...","['as meadows flees from the room , randy fills...",3451_3_3,/content/drive/MyDrive/suraj/thesis/raw_panel_...


In [ ]:
comic_info_df = pd.read_csv('comicnum_to_book_title.csv')
print(comic_info_df.shape)
comic_info_df.dropna(subset=['Book Title', 'Vol'], how='all', inplace=True)
print(comic_info_df.shape)
comic_info_df.head()

(491, 8)
(165, 8)


,comic_no,Book Title,Vol,Issue,Year,Month,genre,link
0,3451,Blue Bolt,10.0,2.0,1949,Sep,superhero|vigilante,NaN
1,3452,Dick Cole - Blue Bolt,3.0,1.0,1949,Jun,superhero|vigilante,NaN
2,3453,Dick Cole - Blue Bolt,5.0,3.0,NaN,Dec,superhero|vigilante,NaN
3,3454,Dick Cole - Blue Bolt,2.0,7.0,NaN,Dec,superhero|vigilante,NaN
4,3455,Dick Cole - Blue Bolt,5.0,7.0,NaN,NaN,superhero|vigilante,NaN


In [ ]:
comic_info_df.tail()

,comic_no,Book Title,Vol,Issue,Year,Month,genre,link
160,3619,Kaanga - Witch Queen of the Hairy Ones,NaN,93.0,1947,Sep,jungle|animal|adventure,https://digitalcomicmuseum.com/index.php?dlid=...
161,3620,kaanga - Vendetta of the Free Tribes,NaN,92.0,1947,Aug,jungle|animal|adventure,https://digitalcomicmuseum.com/index.php?dlid=...
162,3621,JoJo - The Mountain of Skulls,NaN,8.0,1947,Nov,jungle|adventure,https://digitalcomicmuseum.com/index.php?dlid=...
163,3622,JoJo - The Copy-Mad Killers,NaN,28.0,1949,Jun,jungle|adventure,https://digitalcomicmuseum.com/index.php?dlid=...
164,3623,Jumbo Comics - Sheena - Tigerman terror,NaN,26.0,1941,Apr,jungle|adventure|female|scifi|children,https://digitalcomicmuseum.com/index.php?dlid=...


In [ ]:
merged_df = pd.merge(right =comic_info_df, left=raw_text_df, left_on='comic_no', right_on='comic_no', how='inner')
print(merged_df.shape)
merged_df.head()

(50778, 17)


,comic_no,page_no,panel_no,text,concatenated_text,fastpunct_text,decontracted_text,text_clean,image_id,img_file_path,Book Title,Vol,Issue,Year,Month,genre,link
0,3451,2,0,['mercy but the boy he tries to save is too ho...,mercy but the boy he tries to save is too hot ...,"['Mercy, but the boy he tries to save is too h...","['mercy , but the boy he tries to save is too ...","['mercy , but the boy he tries to save is too ...",3451_2_0,/content/drive/MyDrive/suraj/thesis/raw_panel_...,Blue Bolt,10.0,2.0,1949,Sep,superhero|vigilante,NaN
1,3451,3,0,"['nsion several miles from the grove, althy un...","nsion several miles from the grove , althy un...","['Cinder several miles from the grove, along R...","['cinder several miles from the grove , along ...","['cinder several miles from the grove , along ...",3451_3_0,/content/drive/MyDrive/suraj/thesis/raw_panel_...,Blue Bolt,10.0,2.0,1949,Sep,superhero|vigilante,NaN
2,3451,3,1,"['e e owwwi', 'randy expresses his dislike ey ...",e - e - owwwi. randy expresses his dislike ey ...,"['E e owww.i.', 'Randy expresses his dislike a...","['e e owww .i .', 'randy expresses his dislike...","['e e owww .i .', 'randy expresses his dislike...",3451_3_1,/content/drive/MyDrive/suraj/thesis/raw_panel_...,Blue Bolt,10.0,2.0,1949,Sep,superhero|vigilante,NaN
3,3451,3,2,"['so. take one yourself, meadowsd']","so ... take one yourself , meadowsd","['So take one yourself, meadowsd.']","['so take one yourself , meadowsd .']","['so take one yourself , meadowsd .']",3451_3_2,/content/drive/MyDrive/suraj/thesis/raw_panel_...,Blue Bolt,10.0,2.0,1949,Sep,superhero|vigilante,NaN
4,3451,3,3,"['as meadows flees from the room, randy fills ...","as meadows flees from the room , randy fills ...","['As Meadows flees from the room, Randy fills ...","['as meadows flees from the room , randy fills...","['as meadows flees from the room , randy fills...",3451_3_3,/content/drive/MyDrive/suraj/thesis/raw_panel_...,Blue Bolt,10.0,2.0,1949,Sep,superhero|vigilante,NaN


In [ ]:
merged_df.tail()

In [ ]:
print(merged_df.shape)
merged_df.iloc[0,7]

(50778, 17)


"['mercy , but the boy he tries to save is too hot to handle .', 'on summer vacation from farr military aca dem y simba karno and dick camp out in a redwoo grove in california .', 'great idea , coming out here , dick ! it is beautiful !', 'blue button , vol . 10 , no . 2 , september october , 1949 , publish ed bi monthly by the premium group comics , a of curtis circulation company , p . 0 . box 1198 , independence square , philadelphia , pa . offices , we .']"

In [ ]:
def strip_brackets(text_clean):
  text_clean_new = ''
  text_clean_new  = text_clean.replace('[', '')
  text_clean_new  = text_clean_new.replace(']', '')
  text_clean_new  = text_clean_new.replace("'", '')
  text_clean_new  = text_clean_new.replace(".,", '.')

  return text_clean_new

In [ ]:
len(merged_df['comic_no'].unique())

165

In [ ]:
text_per_book_df = merged_df.groupby(['comic_no','Book Title']).agg({'text_clean': lambda x: ' '.join(x)}).reset_index()
text_per_book_df['text_clean_new'] = text_per_book_df['text_clean'].apply(strip_brackets)
text_per_book_df['book_title_clean'] = text_per_book_df['Book Title'].apply(lambda x: re.sub('\W+','_', x.lower() ))
text_per_book_df['unique_id'] = text_per_book_df['comic_no'].astype(str) + '_' + text_per_book_df['book_title_clean']
print(text_per_book_df.shape)
text_per_book_df.head()

(165, 6)


,comic_no,Book Title,text_clean,text_clean_new,book_title_clean,unique_id
0,3451,Blue Bolt,"['mercy , but the boy he tries to save is too ...","mercy , but the boy he tries to save is too ho...",blue_bolt,3451_blue_bolt
1,3452,Dick Cole - Blue Bolt,"['ld cap hawkins , ill .', 'blue bolt , sub ze...","ld cap hawkins , ill . blue bolt , sub zero ed...",dick_cole_blue_bolt,3452_dick_cole_blue_bolt
2,3453,Dick Cole - Blue Bolt,"['eating blue bol .', 'december .'] ['some sho...",eating blue bol . december . some short sighte...,dick_cole_blue_bolt,3453_dick_cole_blue_bolt
3,3454,Dick Cole - Blue Bolt,"['blue , bolt , edison bell is dick cole .'] [...","blue , bolt , edison bell is dick cole . vol ....",dick_cole_blue_bolt,3454_dick_cole_blue_bolt
4,3455,Dick Cole - Blue Bolt,['come and learn .'] ['the basketball season i...,come and learn . the basketball season is in f...,dick_cole_blue_bolt,3455_dick_cole_blue_bolt


In [ ]:
def save_dialogue_as_text(row):

  with open(row['unique_id']+".txt", "w") as text_file:
    text_file.write(row['text_clean_new'])

  row['dialogue_text_filepath'] = os.path.join(dataset_path, row['unique_id']+".txt")
  return row

In [ ]:
text_per_book_df = text_per_book_df.apply(save_dialogue_as_text, axis=1)

In [ ]:
text_per_book_df.head()

,comic_no,Book Title,text_clean,text_clean_new,book_title_clean,unique_id,dialogue_text_filepath
0,3451,Blue Bolt,"['mercy , but the boy he tries to save is too ...","mercy , but the boy he tries to save is too ho...",blue_bolt,3451_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
1,3452,Dick Cole - Blue Bolt,"['ld cap hawkins , ill .', 'blue bolt , sub ze...","ld cap hawkins , ill . blue bolt , sub zero ed...",dick_cole_blue_bolt,3452_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
2,3453,Dick Cole - Blue Bolt,"['eating blue bol .', 'december .'] ['some sho...",eating blue bol . december . some short sighte...,dick_cole_blue_bolt,3453_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
3,3454,Dick Cole - Blue Bolt,"['blue , bolt , edison bell is dick cole .'] [...","blue , bolt , edison bell is dick cole . vol ....",dick_cole_blue_bolt,3454_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
4,3455,Dick Cole - Blue Bolt,['come and learn .'] ['the basketball season i...,come and learn . the basketball season is in f...,dick_cole_blue_bolt,3455_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...


In [ ]:
text_per_book_df.dtypes

comic_no                   int64
Book Title                object
text_clean                object
text_clean_new            object
book_title_clean          object
unique_id                 object
dialogue_text_filepath    object
dtype: object

In [ ]:
text_per_book_df.to_csv('booknlp_text_per_book_df.csv', index=False)

# Initialize book nlp pipeline

In [ ]:
model_params={
		"pipeline":"entity,quote,supersense,event,coref", 
		"model":"big", 
	}

In [ ]:
booknlp=BookNLP("en", model_params)

{'pipeline': 'entity,quote,supersense,event,coref', 'model': 'big'}
downloading entities_google_bert_uncased_L-6_H-768_A-12-v1.0.model
downloading coref_google_bert_uncased_L-12_H-768_A-12-v1.0.model
downloading speaker_google_bert_uncased_L-12_H-768_A-12-v1.0.1.model


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/258M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

--- startup: 67.821 seconds ---


# Run Book NLP Process to extract all entities

In [ ]:
text_per_book_df = pd.read_csv('booknlp_text_per_book_df.csv')
text_per_book_df = text_per_book_df[text_per_book_df['comic_no'] > 3520]
print(text_per_book_df.shape)
text_per_book_df.head()

(95, 7)


,comic_no,Book Title,text_clean,text_clean_new,book_title_clean,unique_id,dialogue_text_filepath
70,3521,Daring Adventures - Dynamic man meets the litt...,['suched the grisly answer although knew he wo...,suched the grisly answer although knew he woul...,daring_adventures_dynamic_man_meets_the_little...,3521_daring_adventures_dynamic_man_meets_the_l...,/content/drive/MyDrive/suraj/thesis/dataset/em...
71,3522,Dynamic Man- The Echo,['comic .'] ['the dreams of dozens of poor boy...,comic . the dreams of dozens of poor boys were...,dynamic_man_the_echo,3522_dynamic_man_the_echo,/content/drive/MyDrive/suraj/thesis/dataset/em...
72,3523,Dynamic Man - man on the run,"[""what defiant urges zao a s betzine , the unw...","""what defiant urges zao a s betzine , the unwi...",dynamic_man_man_on_the_run,3523_dynamic_man_man_on_the_run,/content/drive/MyDrive/suraj/thesis/dataset/em...
73,3524,The Eagle - Monster of tropics,['before the nazis could open fire on the red ...,before the nazis could open fire on the red cr...,the_eagle_monster_of_tropics,3524_the_eagle_monster_of_tropics,/content/drive/MyDrive/suraj/thesis/dataset/em...
74,3525,Exciting Comics - Black terror - Malay Menace,"['eating the black terror .', 'black terror , ...","eating the black terror . black terror , immed...",exciting_comics_black_terror_malay_menace,3525_exciting_comics_black_terror_malay_menace,/content/drive/MyDrive/suraj/thesis/dataset/em...


In [ ]:
def extract_information_from_dialogues(row):
  print(str(row.name) + ' ' + row['unique_id'])
  inputFile = row['dialogue_text_filepath']
  outputDir = row['unique_id'] + "/"
  idd = row['unique_id']
  booknlp.process(inputFile, outputDir, idd)
  print(' ============= ============== ============== ============ ')
  print()

  return row


In [ ]:
text_per_book_df = text_per_book_df.apply(extract_information_from_dialogues, axis=1)

70 3521_daring_adventures_dynamic_man_meets_the_little_people
--- spacy: 0.540 seconds ---
--- entities: 12.406 seconds ---
--- quotes: 0.008 seconds ---
--- attribution: 2.267 seconds ---
--- name coref: 0.007 seconds ---
--- coref: 19.672 seconds ---
--- TOTAL (excl. startup): 35.611 seconds ---, 3827 words
 ============= ============== ============== ============ 

71 3522_dynamic_man_the_echo
--- spacy: 1.033 seconds ---
--- entities: 29.585 seconds ---
--- quotes: 0.017 seconds ---
--- attribution: 4.454 seconds ---
--- name coref: 0.023 seconds ---
--- coref: 36.634 seconds ---
--- TOTAL (excl. startup): 72.405 seconds ---, 8120 words
 ============= ============== ============== ============ 

72 3523_dynamic_man_man_on_the_run
--- spacy: 0.731 seconds ---
--- entities: 17.556 seconds ---
--- quotes: 0.013 seconds ---
--- attribution: 6.843 seconds ---
--- name coref: 0.016 seconds ---
--- coref: 26.047 seconds ---
--- TOTAL (excl. startup): 51.269 seconds ---, 5671 words
 ======

--- spacy: 1.032 seconds ---
--- entities: 10.091 seconds ---
--- quotes: 0.018 seconds ---
--- attribution: 0.837 seconds ---
--- name coref: 0.013 seconds ---
--- coref: 6.804 seconds ---
--- TOTAL (excl. startup): 19.103 seconds ---, 8083 words


# Evaluate output

In [ ]:
text_per_book_df = pd.read_csv('booknlp_text_per_book_df.csv')
print(text_per_book_df.shape)
text_per_book_df.head()

(165, 7)


,comic_no,Book Title,text_clean,text_clean_new,book_title_clean,unique_id,dialogue_text_filepath
0,3451,Blue Bolt,"['mercy , but the boy he tries to save is too ...","mercy , but the boy he tries to save is too ho...",blue_bolt,3451_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
1,3452,Dick Cole - Blue Bolt,"['ld cap hawkins , ill .', 'blue bolt , sub ze...","ld cap hawkins , ill . blue bolt , sub zero ed...",dick_cole_blue_bolt,3452_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
2,3453,Dick Cole - Blue Bolt,"['eating blue bol .', 'december .'] ['some sho...",eating blue bol . december . some short sighte...,dick_cole_blue_bolt,3453_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
3,3454,Dick Cole - Blue Bolt,"['blue , bolt , edison bell is dick cole .'] [...","blue , bolt , edison bell is dick cole . vol ....",dick_cole_blue_bolt,3454_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
4,3455,Dick Cole - Blue Bolt,['come and learn .'] ['the basketball season i...,come and learn . the basketball season is in f...,dick_cole_blue_bolt,3455_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...


In [ ]:
def proc(filename):
    with open(filename) as file:
        data=json.load(file)
    return data

In [ ]:
data=proc("3452_dick_cole_blue_bolt/3452_dick_cole_blue_bolt.book")

In [ ]:
def get_counter_from_dependency_list(dep_list):
    counter=Counter()
    for token in dep_list:
        term=token["w"]
        tokenGlobalIndex=token["i"]
        counter[term]+=1
    return counter

In [ ]:
dick_cole_blue_bolt_entity_3452_df = pd.read_csv("./3452_dick_cole_blue_bolt/3452_dick_cole_blue_bolt.entities", sep='\t')
print(dick_cole_blue_bolt_entity_3452_df.shape)
dick_cole_blue_bolt_entity_3452_df.head(100)

(1661, 6)


,COREF,start_token,end_token,prop,cat,text
0,50,0,12,NOM,PER,"ld cap hawkins , ill . blue bolt , sub zero ed..."
1,51,22,22,PRON,PER,your
2,52,34,34,PRON,PER,his
3,8,55,57,PROP,PER,john dal marines
4,53,59,59,PRON,PER,us
...,...,...,...,...,...,...
95,8,464,464,PRON,PER,he
96,66,466,466,NOM,PER,indians
97,8,469,469,PRON,PER,he
98,66,471,471,NOM,PER,indians


In [ ]:
for character in data["characters"]:
    
    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]
    
    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
        
        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=10
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))        
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))       
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))    
        print()

8 43 john he/him/his

	poss	3 father
	poss	1 shoulder
	poss	1 work
	poss	1 own
	poss	1 sons
	poss	1 boys

	agent	1 told
	agent	1 comes
	agent	1 studied
	agent	1 tell


	mod	5 private
	mod	1 married
	mod	1 sort
	mod	1 hobby

10 25 dick he/him/his

	poss	1 face
	poss	1 pursuers
	poss	1 puckers
	poss	1 head

	agent	1 starts
	agent	1 picks
	agent	1 eudes
	agent	1 hides
	agent	1 joins
	agent	1 used
	agent	1 accomplished
	agent	1 typifies
	agent	1 reach
	agent	1 started

	patient	1 fix

	mod	2 right

14 21 eddie he/him/his


	agent	2 has
	agent	1 take
	agent	1 sets
	agent	1 flattens
	agent	1 finds
	agent	1 hears
	agent	1 discovered

	patient	1 sends
	patient	1 see
	patient	1 got
	patient	1 find

	mod	1 about

42 14 tito he/him/his

	poss	1 pursler
	poss	1 guns

	agent	1 held
	agent	1 smi
	agent	1 get
	agent	1 go

	patient	1 head
	patient	1 jump

	mod	1 bandit
	mod	1 smarter

19 11 krubb he/him/his

	poss	1 men

	agent	1 takes
	agent	1 dropped
	agent	1 walk
	agent	1 says
	agent	1 leave
	agent

In [ ]:
for character in data["characters"]:
    
    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]
    
    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
        
        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=10
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))        
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))       
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))    
        print()

5 29 steve he/him/his

	poss	2 pursuers
	poss	1 ick
	poss	1 squadron
	poss	1 tv
	poss	1 course
	poss	1 wings
	poss	1 intentions

	agent	1 escorted
	agent	1 sparked
	agent	1 given
	agent	1 return
	agent	1 takes
	agent	1 reversed
	agent	1 striving
	agent	1 wags
	agent	1 tries
	agent	1 knocked

	patient	1 shot
	patient	1 forced


7 2 randy oconnor he/him/his

	poss	1 allies

	agent	1 observed





In [ ]:
for character in data["characters"]:
    
    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]
    
    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
        
        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=10
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))        
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))       
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))    
        print()

5 22 buster crabbe he/him/his


	agent	1 got
	agent	1 showing
	agent	1 git
	agent	1 taking
	agent	1 ai
	agent	1 put
	agent	1 sees
	agent	1 draws
	agent	1 pull
	agent	1 go

	patient	1 killed
	patient	1 silence
	patient	1 learned
	patient	1 beat
	patient	1 get
	patient	1 got

	mod	1 friend
	mod	1 dead

10 5 minnie boo he/him/his


	agent	1 had
	agent	1 wuz
	agent	1 tromped
	agent	1 massacred



7 4 benson he/him/his

	poss	1 gang

	agent	1 call

	patient	1 locked
	patient	1 tell


8 3 .the coroanches they/them/their


	agent	1 knew



6 2 jason he/him/his







In [ ]:
for character in data["characters"]:
    
    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]
    
    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
        
        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=10
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))        
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))       
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))    
        print()

6 13 bryan craig he/him/his

	poss	1 alley
	poss	1 ag
	poss	1 light

	agent	1 like
	agent	1 did
	agent	1 hateem

	patient	1 lost
	patient	1 get
	patient	1 tell
	patient	1 atomize
	patient	1 gaught


13 9 miss tracy she/her

	poss	1 soldier

	agent	1 avenge

	patient	1 tell

	mod	2 spy
	mod	1 help
	mod	1 human

4 8 gordon he/him/his


	agent	1 keep



14 8 krain he/him/his


	agent	1 turns
	agent	1 decides
	agent	1 attracts
	agent	1 hidden
	agent	1 snaps
	agent	1 lands



10 3 captain schenge he/him/his

	poss	2 belt

	agent	1 takes



5 2 trevor r he/him/his

	poss	1 assistant




15 2 frank he/him/his



	patient	1 save




In [ ]:
for character in data["characters"]:
    
    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]
    
    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
        
        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=10
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))        
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))       
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))    
        print()

3 18 brenda she/her

	poss	1 plane
	poss	1 power

	agent	1 recovers
	agent	1 has
	agent	1 drink
	agent	1 got
	agent	1 find
	agent	1 rested
	agent	1 hear

	patient	1 ntranced

	mod	1 lucky
	mod	1 startled

8 6 tom he/him/his


	agent	1 soars
	agent	1 got

	patient	1 dragging


7 5 miss starr she/her

	poss	1 innocence


	patient	1 stop


10 3 guido marcanni he/him/his







# Create male femal prominence, character emotion profile and plot complexity

- plot complexity is number of characters in book
- male female ratio is amount of female characters to male character

In [ ]:
text_per_book_df = pd.read_csv('booknlp_text_per_book_df.csv')
print(text_per_book_df.shape)
text_per_book_df.head()

(165, 7)


,comic_no,Book Title,text_clean,text_clean_new,book_title_clean,unique_id,dialogue_text_filepath
0,3451,Blue Bolt,"['mercy , but the boy he tries to save is too ...","mercy , but the boy he tries to save is too ho...",blue_bolt,3451_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
1,3452,Dick Cole - Blue Bolt,"['ld cap hawkins , ill .', 'blue bolt , sub ze...","ld cap hawkins , ill . blue bolt , sub zero ed...",dick_cole_blue_bolt,3452_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
2,3453,Dick Cole - Blue Bolt,"['eating blue bol .', 'december .'] ['some sho...",eating blue bol . december . some short sighte...,dick_cole_blue_bolt,3453_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
3,3454,Dick Cole - Blue Bolt,"['blue , bolt , edison bell is dick cole .'] [...","blue , bolt , edison bell is dick cole . vol ....",dick_cole_blue_bolt,3454_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
4,3455,Dick Cole - Blue Bolt,['come and learn .'] ['the basketball season i...,come and learn . the basketball season is in f...,dick_cole_blue_bolt,3455_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...


In [ ]:
def proc(filename):
    with open(filename) as file:
        data=json.load(file)
    return data

In [ ]:
def get_counter_from_dependency_list(dep_list):
    counter=Counter()
    for token in dep_list:
        term=token["w"]
        tokenGlobalIndex=token["i"]
        counter[term]+=1
    return counter

In [ ]:
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


In [ ]:
data=proc("3452_dick_cole_blue_bolt/3452_dick_cole_blue_bolt.book")

In [ ]:
def get_character_information(row):
    
    

    # read the book json
    data = proc('{}/{}.book'.format(row['unique_id'], row['unique_id']))
    comic_no = row['unique_id'].split('_')[0]
    male_orientation = 0.0
    female_orientation = 0.0
    neutral_orientation = 0.0
    unique_character_count = 0
    emotion_profile = None

    character_dict = {comic_no: []} #  {'name': '', 'proper_mention_count': 0, 'all_mention_count': 0,,'gender': 'unknown', 'agent': [], 'patient': [], 'modifier': [], 'possession: []'}

    # loop over each character
    for character in data["characters"]:
      
    
      agentList=character["agent"]
      patientList=character["patient"]
      possList=character["poss"]
      modList=character["mod"]
      
      character_id=character["id"]
      count=character["count"]

      referential_gender_distribution=referential_gender_prediction="unknown"

      if character["g"] is not None and character["g"] != "unknown":
          referential_gender_distribution=character["g"]["inference"]
          referential_gender=character["g"]["argmax"]

      mentions = character["mentions"]
      proper_mention = mentions["proper"]
      common_mentions = mentions["common"] 
      pronoun_mentions =  mentions["pronoun"] 
      max_proper_mention=""
      printTop=10
      
      # just print out information about named characters
      if len(mentions["proper"]) > 0:
          unique_character_count += 1
          max_proper_mention=mentions["proper"][0]["n"]
          # print(character_id, count, max_proper_mention, referential_gender)

          if referential_gender == 'he/him/his':
            male_orientation += count
          elif referential_gender == 'she/her':
            female_orientation += count
          elif referential_gender == 'they/them/their':
            neutral_orientation += count

          agent_word_lst = [k for k,v in get_counter_from_dependency_list(agentList).most_common(printTop)]
          patient_word_lst = [k for k,v in get_counter_from_dependency_list(patientList).most_common(printTop)]
          poss_word_lst = [k for k,v in get_counter_from_dependency_list(possList).most_common(printTop)]
          mod_word_lst = [k for k,v in get_counter_from_dependency_list(modList).most_common(printTop)]

          # emotion profile of character using simple NRC
          lst_of_keywords = agent_word_lst + patient_word_lst + poss_word_lst + mod_word_lst
          keyword_str = ' '.join(lst_of_keywords)
          emo_object = NRCLex(keyword_str)
          emotion_profile = emo_object.affect_frequencies

          
          character_dict[comic_no].append({'name': max_proper_mention, 'character_id': character_id, 'mention_count': count, 'gender': referential_gender, 
                                           'agent':  agent_word_lst, 
                                           'patient':  patient_word_lst, 
                                           'modifier':  mod_word_lst, 
                                           'possession': poss_word_lst,
                                           'emotion_profile': emotion_profile})
          
    
    row['character_dict'] = character_dict
    row['unique_character_count'] = unique_character_count
    row['male_orientation_count'] = male_orientation
    row['female_orientation_count'] = female_orientation
    row['neutral_orientation_count'] = neutral_orientation
    row['male_orientation_ratio'] = male_orientation/(male_orientation+female_orientation+neutral_orientation+1e-6)
    row['female_orientation_ratio'] = female_orientation/(male_orientation+female_orientation+neutral_orientation+1e-6)
    row['neutral_orientation_ratio'] = neutral_orientation/(male_orientation+female_orientation+neutral_orientation+1e-6)


    return row

In [ ]:
text_per_book_df = text_per_book_df.apply(get_character_information, axis=1)

In [ ]:
text_per_book_df.head(20)

In [ ]:
text_per_book_df.to_csv('text_per_book_df_20220619.csv', index=False)

In [ ]:
text_per_book_df[text_per_book_df['comic_no'] == 3463]['character_dict'].values

array([{'3463': [{'name': 'brenda', 'character_id': 3, 'mention_count': 49, 'gender': 'she/her', 'agent': ['know', 'go', 'recovers', 'arrived', 'excuse', 'getting', 'let', 'has', 'offered', 'got'], 'patient': ['see', 'kiss', 'ntranced', 'take', 'thank', 'sending', 'held', 'stop'], 'modifier': ['interesting', 'sad', 'startled'], 'possession': ['things', 'work', 'suit', 'service', 'week', 'boss', 'innocence', 'power'], 'emotion_profile': {'negative': 1, 'anticipation': 1, 'joy': 1, 'positive': 3, 'surprise': 1}}, {'name': 'pesky', 'character_id': 12, 'mention_count': 9, 'gender': 'he/him/his', 'agent': ['goes', 'hear', 'gone', 'removes'], 'patient': ['call'], 'modifier': [], 'possession': ['ystery'], 'emotion_profile': {}}, {'name': 'tom', 'character_id': 11, 'mention_count': 8, 'gender': 'he/him/his', 'agent': ['wires', 'think'], 'patient': ['removes', 'dragging'], 'modifier': ['unable'], 'possession': [], 'emotion_profile': {'negative': 1, 'sadness': 1}}, {'name': 'larry nickels', 'cha

In [ ]:
te.get_emotion("""father, owner, shoulder, work, own, sons, boys, told, comes, studied, tell, private, marries, sort, hobby """)

{'Angry': 0.0, 'Fear': 0.25, 'Happy': 0.0, 'Sad': 0.5, 'Surprise': 0.25}

In [ ]:
token_lst = ['father', 'owner', 'shoulder','work', 'own', 'sons', 'boys', 'told', 'comes', 'studied', 'tell', 'private', 'marries', 'sort', 'hobby']
text_object = NRCLex("""father, owner, shoulder, work, own, sons, boys, told, comes, studied, tell, private, marries, sort, hobby """)
print(text_object.affect_frequencies)
print(text_object.raw_emotion_scores)

{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 'trust': 0.4, 'surprise': 0.0, 'positive': 0.4, 'negative': 0.0, 'sadness': 0.0, 'disgust': 0.0, 'joy': 0.2}
{'trust': 2, 'positive': 2, 'joy': 1}


# supersense tagging

In [ ]:
text_per_book_df = pd.read_csv('text_per_book_df_20220619.csv')
print(text_per_book_df.shape)
text_per_book_df.head()

(165, 15)


,comic_no,Book Title,text_clean,text_clean_new,book_title_clean,unique_id,dialogue_text_filepath,character_dict,unique_character_count,male_orientation_count,female_orientation_count,neutral_orientation_count,male_orientation_ratio,female_orientation_ratio,neutral_orientation_ratio
0,3451,Blue Bolt,"['mercy , but the boy he tries to save is too ...","mercy , but the boy he tries to save is too ho...",blue_bolt,3451_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3451': [{'name': 'randy', 'character_id': 7,...",5,33.0,2.0,2.0,0.891892,0.054054,0.054054
1,3452,Dick Cole - Blue Bolt,"['ld cap hawkins , ill .', 'blue bolt , sub ze...","ld cap hawkins , ill . blue bolt , sub zero ed...",dick_cole_blue_bolt,3452_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3452': [{'name': 'john', 'character_id': 8, ...",23,185.0,5.0,2.0,0.963542,0.026042,0.010417
2,3453,Dick Cole - Blue Bolt,"['eating blue bol .', 'december .'] ['some sho...",eating blue bol . december . some short sighte...,dick_cole_blue_bolt,3453_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3453': [{'name': 'eddie', 'character_id': 9,...",18,106.0,0.0,7.0,0.938053,0.000000,0.061947
3,3454,Dick Cole - Blue Bolt,"['blue , bolt , edison bell is dick cole .'] [...","blue , bolt , edison bell is dick cole . vol ....",dick_cole_blue_bolt,3454_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3454': [{'name': 'simba', 'character_id': 7,...",15,71.0,6.0,0.0,0.922078,0.077922,0.000000
4,3455,Dick Cole - Blue Bolt,['come and learn .'] ['the basketball season i...,come and learn . the basketball season is in f...,dick_cole_blue_bolt,3455_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3455': [{'name': 'dick', 'character_id': 6, ...",14,94.0,8.0,2.0,0.903846,0.076923,0.019231


In [ ]:
supersense_tags_df = pd.read_csv('supersense_tagging.csv')
supersense_tags_df.dropna(subset=['supersense_tag_type'], inplace=True)
supersense_tags_lst = list(supersense_tags_df['supersense_tag_type'].unique())
supersense_tags_dict = {k: 0.0 for k in supersense_tags_lst}
print(supersense_tags_df.shape)
supersense_tags_df.head()

(45, 11)


,sl_no,supersense_tag_type,type,misc1,misc2,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,0,adj.all,all,adjective,clusters,NaN,NaN,NaN,NaN,NaN,NaN
1,1,adj.pert,relational,adjectives,(pertainyms),NaN,NaN,NaN,NaN,NaN,NaN
2,2,adv.all,all,adverbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,noun.Tops,unique,beginner,for,nouns,NaN,NaN,NaN,NaN,NaN
4,4,noun.act,nouns,denoting,acts,or,actions,NaN,NaN,NaN,NaN


In [ ]:
supersense_tags_dict

{'adj.all': 0.0,
 'adj.pert': 0.0,
 'adj.ppl': 0.0,
 'adv.all': 0.0,
 'noun.Tops': 0.0,
 'noun.act': 0.0,
 'noun.animal': 0.0,
 'noun.artifact': 0.0,
 'noun.attribute': 0.0,
 'noun.body': 0.0,
 'noun.cognition': 0.0,
 'noun.communication': 0.0,
 'noun.event': 0.0,
 'noun.feeling': 0.0,
 'noun.food': 0.0,
 'noun.group': 0.0,
 'noun.location': 0.0,
 'noun.motive': 0.0,
 'noun.object': 0.0,
 'noun.person': 0.0,
 'noun.phenomenon': 0.0,
 'noun.plant': 0.0,
 'noun.possession': 0.0,
 'noun.process': 0.0,
 'noun.quantity': 0.0,
 'noun.relation': 0.0,
 'noun.shape': 0.0,
 'noun.state': 0.0,
 'noun.substance': 0.0,
 'noun.time': 0.0,
 'verb.body': 0.0,
 'verb.change': 0.0,
 'verb.cognition': 0.0,
 'verb.communication': 0.0,
 'verb.competition': 0.0,
 'verb.consumption': 0.0,
 'verb.contact': 0.0,
 'verb.creation': 0.0,
 'verb.emotion': 0.0,
 'verb.motion': 0.0,
 'verb.perception': 0.0,
 'verb.possession': 0.0,
 'verb.social': 0.0,
 'verb.stative': 0.0,
 'verb.weather': 0.0}

In [ ]:
# add super sense tags rows
for k,v in supersense_tags_dict.items():
    text_per_book_df[k+'_count'] = 0.0
    text_per_book_df[k+'_ratio'] = 0.0

text_per_book_df.head()

,comic_no,Book Title,text_clean,text_clean_new,book_title_clean,unique_id,dialogue_text_filepath,character_dict,unique_character_count,male_orientation_count,...,verb.possession_count,verb.possession_ratio,verb.social_count,verb.social_ratio,verb.stative_count,verb.stative_ratio,verb.weather_count,verb.weather_ratio,adj.ppl_count,adj.ppl_ratio
0,3451,Blue Bolt,"['mercy , but the boy he tries to save is too ...","mercy , but the boy he tries to save is too ho...",blue_bolt,3451_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3451': [{'name': 'randy', 'character_id': 7,...",5,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3452,Dick Cole - Blue Bolt,"['ld cap hawkins , ill .', 'blue bolt , sub ze...","ld cap hawkins , ill . blue bolt , sub zero ed...",dick_cole_blue_bolt,3452_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3452': [{'name': 'john', 'character_id': 8, ...",23,185.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3453,Dick Cole - Blue Bolt,"['eating blue bol .', 'december .'] ['some sho...",eating blue bol . december . some short sighte...,dick_cole_blue_bolt,3453_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3453': [{'name': 'eddie', 'character_id': 9,...",18,106.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3454,Dick Cole - Blue Bolt,"['blue , bolt , edison bell is dick cole .'] [...","blue , bolt , edison bell is dick cole . vol ....",dick_cole_blue_bolt,3454_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3454': [{'name': 'simba', 'character_id': 7,...",15,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3455,Dick Cole - Blue Bolt,['come and learn .'] ['the basketball season i...,come and learn . the basketball season is in f...,dick_cole_blue_bolt,3455_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3455': [{'name': 'dick', 'character_id': 6, ...",14,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_supersense = pd.read_csv('{}/{}.supersense'.format('3591_hit_comics_keeper_kid_eternity_the_highness', '3591_hit_comics_keeper_kid_eternity_the_highness'), delimiter="\t", error_bad_lines=False, quotechar="\t")

In [ ]:
def get_supersense_info(row):
    print(row.name, row['unique_id'])
    df_supersense = pd.read_csv('{}/{}.supersense'.format(row['unique_id'], row['unique_id']), delimiter="\t", error_bad_lines=False, quotechar="\t")
    grouped_df = df_supersense.groupby(['supersense_category'])['supersense_category'].count().reset_index(name='supersense_count')
    grouped_df['total_supersense_tags'] = grouped_df['supersense_count'].sum()
    grouped_df['supersense_tag_ratio'] = grouped_df['supersense_count']/grouped_df['total_supersense_tags'] 

    supersense_count_dict = dict(grouped_df[['supersense_category', 'supersense_count']].values)
    supersense_ratio_dict = dict(grouped_df[['supersense_category', 'supersense_tag_ratio']].values)


    # add ratio values
    for k,v in supersense_ratio_dict.items():
        row[k+'_ratio'] = v

    # add count values
    for k,v in supersense_count_dict.items():
        row[k+'_count'] = v

    return row

In [ ]:
text_per_book_df = text_per_book_df.apply(get_supersense_info, axis=1)
text_per_book_df.head()

0 3451_blue_bolt
1 3452_dick_cole_blue_bolt
2 3453_dick_cole_blue_bolt
3 3454_dick_cole_blue_bolt
4 3455_dick_cole_blue_bolt
5 3456_blue_bolt
6 3457_dick_cole_blue_bolt
7 3458_lightning_man_blue_bolt
8 3459_barb_riders
9 3460_straight_arrow
10 3461_boy_crimebuster
11 3462_boy_detective
12 3463_brenda_starr_silver_lining_in_sun_valley
13 3464_brenda_starr_shopping_for_trouble
14 3465_brenda_starr_the_great_penicillin_plot
15 3466_brenda_starr_assignment_preferred
16 3467_broncho_bill_justice_on_the_range
17 3468_black_terror_the_dream_maker
18 3469_buster_crabbe_and_the_maid_of_mars
19 3470_captain_marvel_jr_bird_of_doom
20 3471_captain_marvel_jr_the_hunt_for_treasure
21 3472_captain_marvel_jr_the_acrobat_death_trap
22 3473_captain_marvel_jr_voodoo_clock
23 3474_captain_marvel_jr_keep_the_liberty_bell_ringing
24 3475_captain_marvel_jr_irresistible_sword
25 3476_captain_science_space_pirates
26 3477_captain_science_mental_monsters
27 3478_captain_science_martian_salvers
28 3479_captain_s

,comic_no,Book Title,text_clean,text_clean_new,book_title_clean,unique_id,dialogue_text_filepath,character_dict,unique_character_count,male_orientation_count,...,verb.possession_count,verb.possession_ratio,verb.social_count,verb.social_ratio,verb.stative_count,verb.stative_ratio,verb.weather_count,verb.weather_ratio,adj.ppl_count,adj.ppl_ratio
0,3451,Blue Bolt,"['mercy , but the boy he tries to save is too ...","mercy , but the boy he tries to save is too ho...",blue_bolt,3451_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3451': [{'name': 'randy', 'character_id': 7,...",5,33.0,...,44,0.037801,40,0.034364,113,0.097079,1.0,0.000859,0.0,0.0
1,3452,Dick Cole - Blue Bolt,"['ld cap hawkins , ill .', 'blue bolt , sub ze...","ld cap hawkins , ill . blue bolt , sub zero ed...",dick_cole_blue_bolt,3452_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3452': [{'name': 'john', 'character_id': 8, ...",23,185.0,...,108,0.030058,132,0.036738,332,0.092402,2.0,0.000557,0.0,0.0
2,3453,Dick Cole - Blue Bolt,"['eating blue bol .', 'december .'] ['some sho...",eating blue bol . december . some short sighte...,dick_cole_blue_bolt,3453_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3453': [{'name': 'eddie', 'character_id': 9,...",18,106.0,...,90,0.035517,73,0.028808,246,0.097080,1.0,0.000395,0.0,0.0
3,3454,Dick Cole - Blue Bolt,"['blue , bolt , edison bell is dick cole .'] [...","blue , bolt , edison bell is dick cole . vol ....",dick_cole_blue_bolt,3454_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3454': [{'name': 'simba', 'character_id': 7,...",15,71.0,...,77,0.026014,120,0.040541,270,0.091216,1.0,0.000338,0.0,0.0
4,3455,Dick Cole - Blue Bolt,['come and learn .'] ['the basketball season i...,come and learn . the basketball season is in f...,dick_cole_blue_bolt,3455_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3455': [{'name': 'dick', 'character_id': 6, ...",14,94.0,...,80,0.035119,95,0.041703,183,0.080334,1.0,0.000439,0.0,0.0


In [ ]:
text_per_book_df.to_csv('text_per_book_df_20220619.csv', index=False)

In [ ]:
all_col_lst = list(text_per_book_df.columns)
del all_col_lst[2:4]


In [ ]:
# save only features without text
text_per_book_df[all_col_lst].to_csv('features_supersense_character_gender_df_20220619.csv', index=False)

In [ ]:
text_per_book_df[all_col_lst].head(20)

,comic_no,Book Title,book_title_clean,unique_id,dialogue_text_filepath,character_dict,unique_character_count,male_orientation_count,female_orientation_count,neutral_orientation_count,...,verb.possession_count,verb.possession_ratio,verb.social_count,verb.social_ratio,verb.stative_count,verb.stative_ratio,verb.weather_count,verb.weather_ratio,adj.ppl_count,adj.ppl_ratio
0,3451,Blue Bolt,blue_bolt,3451_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3451': [{'name': 'randy', 'character_id': 7,...",5,33.0,2.0,2.0,...,44,0.037801,40,0.034364,113,0.097079,1.0,0.000859,0.0,0.0
1,3452,Dick Cole - Blue Bolt,dick_cole_blue_bolt,3452_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3452': [{'name': 'john', 'character_id': 8, ...",23,185.0,5.0,2.0,...,108,0.030058,132,0.036738,332,0.092402,2.0,0.000557,0.0,0.0
2,3453,Dick Cole - Blue Bolt,dick_cole_blue_bolt,3453_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3453': [{'name': 'eddie', 'character_id': 9,...",18,106.0,0.0,7.0,...,90,0.035517,73,0.028808,246,0.097080,1.0,0.000395,0.0,0.0
3,3454,Dick Cole - Blue Bolt,dick_cole_blue_bolt,3454_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3454': [{'name': 'simba', 'character_id': 7,...",15,71.0,6.0,0.0,...,77,0.026014,120,0.040541,270,0.091216,1.0,0.000338,0.0,0.0
4,3455,Dick Cole - Blue Bolt,dick_cole_blue_bolt,3455_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3455': [{'name': 'dick', 'character_id': 6, ...",14,94.0,8.0,2.0,...,80,0.035119,95,0.041703,183,0.080334,1.0,0.000439,0.0,0.0
5,3456,Blue Bolt,blue_bolt,3456_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3456': [{'name': 'cowboy jim bowers', 'chara...",17,67.0,4.0,0.0,...,84,0.039792,81,0.038370,206,0.097584,0.0,0.000000,0.0,0.0
6,3457,Dick Cole - Blue Bolt,dick_cole_blue_bolt,3457_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3457': [{'name': 'gelau', 'character_id': 16...",12,65.0,2.0,0.0,...,86,0.040566,67,0.031604,207,0.097642,0.0,0.000000,0.0,0.0
7,3458,Lightning Man - Blue Bolt,lightning_man_blue_bolt,3458_lightning_man_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3458': [{'name': 'peter dawn', 'character_id...",11,36.0,7.0,8.0,...,63,0.026912,93,0.039727,201,0.085861,1.0,0.000427,0.0,0.0
8,3459,Barb Riders,barb_riders,3459_barb_riders,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3459': [{'name': 'pedro hernandez', 'charact...",3,14.0,0.0,0.0,...,61,0.048374,55,0.043616,94,0.074544,0.0,0.000000,0.0,0.0
9,3460,Straight Arrow,straight_arrow,3460_straight_arrow,/content/drive/MyDrive/suraj/thesis/dataset/em...,"{'3460': [{'name': 'kate collins', 'character_...",2,7.0,0.0,3.0,...,53,0.035475,62,0.041499,113,0.075636,0.0,0.000000,0.0,0.0


# Personality Trait detection

In [ ]:
 mylst = ['know', 'go', 'recovers', 'arrived', 'excuse', 'getting', 'let', 'has', 'offer', 'got'] + ['see', 'kiss', 'trance', 'take', 'thank', 'sending', 'held', 'stop'] + ['interesting', 'sad', 'startled'] + ['things', 'work', 'suit', 'service', 'week', 'boss', 'innocence', 'power']
 mytxt = " ".join(mylst)

In [ ]:
cEXT = pickle.load( open( "personality_traits/models/cEXT.p", "rb"))
cNEU = pickle.load( open( "personality_traits/models/cNEU.p", "rb"))
cAGR = pickle.load( open( "personality_traits/models/cAGR.p", "rb"))
cCON = pickle.load( open( "personality_traits/models/cCON.p", "rb"))
cOPN = pickle.load( open( "personality_traits/models/cOPN.p", "rb"))
vectorizer_31 = pickle.load( open( "personality_traits/models/vectorizer_31.p", "rb"))
vectorizer_30 = pickle.load( open( "personality_traits/models/vectorizer_30.p", "rb"))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.1 when using version 1.0.2. This might lead to breaking code or invalid resul

In [ ]:
def predict_personality(text):
    scentences = re.split("(?<=[.!?]) +", text)
    text_vector_31 = vectorizer_31.transform(scentences)
    text_vector_30 = vectorizer_30.transform(scentences)
    EXT = cEXT.predict(text_vector_31)
    NEU = cNEU.predict(text_vector_30)
    AGR = cAGR.predict(text_vector_31)
    CON = cCON.predict(text_vector_31)
    OPN = cOPN.predict(text_vector_31)
    print(EXT, NEU, AGR, CON, OPN)
    return [EXT[0], NEU[0], AGR[0], CON[0], OPN[0]]

In [ ]:
sample_text = """Do I really look like a guy with a plan? You know what I am? I'm a dog chasing cars. I wouldn't know what to do with one if I caught it! You know, I just DO things."""
# sample_text = """ Wanna know how I got these scars? My father was a drinker. And a fiend. And one night he goes off crazier than usual. Mommy gets the kitchen knife to defend herself. He doesn’t like that. Not one bit. So—me watching—he takes the knife to her, laughing while he does it! Turns to me, and he says, “Why so serious, son?” Comes at me with the knife.“Why so serious?” He sticks the blade in my mouth…“Let’s put a smile on that face!” """

In [ ]:
mytxt

'know go recovers arrived excuse getting let has offer got see kiss trance take thank sending held stop interesting sad startled things work suit service week boss innocence power'

In [ ]:
predictions = predict_personality(sample_text)
print("predicted personality:", predictions)

[1 1 0 1 1] [1 1 1 1 1] [1 0 0 1 1] [0 0 0 0 0] [1 1 1 1 1]
predicted personality: [1, 1, 1, 0, 1]


# Miscellaneous

In [ ]:
for character in data["characters"]:
    
    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]
    
    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
        
        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=10
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))        
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))       
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))    
        print()

8 43 john he/him/his

	poss	3 father
	poss	1 shoulder
	poss	1 work
	poss	1 own
	poss	1 sons
	poss	1 boys

	agent	1 told
	agent	1 comes
	agent	1 studied
	agent	1 tell


	mod	5 private
	mod	1 married
	mod	1 sort
	mod	1 hobby

10 25 dick he/him/his

	poss	1 face
	poss	1 pursuers
	poss	1 puckers
	poss	1 head

	agent	1 starts
	agent	1 picks
	agent	1 eudes
	agent	1 hides
	agent	1 joins
	agent	1 used
	agent	1 accomplished
	agent	1 typifies
	agent	1 reach
	agent	1 started

	patient	1 fix

	mod	2 right

14 21 eddie he/him/his


	agent	2 has
	agent	1 take
	agent	1 sets
	agent	1 flattens
	agent	1 finds
	agent	1 hears
	agent	1 discovered

	patient	1 sends
	patient	1 see
	patient	1 got
	patient	1 find

	mod	1 about

42 14 tito he/him/his

	poss	1 pursler
	poss	1 guns

	agent	1 held
	agent	1 smi
	agent	1 get
	agent	1 go

	patient	1 head
	patient	1 jump

	mod	1 bandit
	mod	1 smarter

19 11 krubb he/him/his

	poss	1 men

	agent	1 takes
	agent	1 dropped
	agent	1 walk
	agent	1 says
	agent	1 leave
	agent

In [ ]:
# company_names = names['Company Name']
# vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
# tf_idf_matrix = vectorizer.fit_transform(company_names)

"""
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))


"""

"""
import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1
print("SELFTIMED:", t)
"""

"""
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})
"""

"""
matches_df = get_matches_df(matches, company_names, top=100000)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # Remove all exact matches
matches_df.sample(20)
"""

In [ ]:
spell = Speller(lang='en')


In [ ]:
spell('mr. ronson')

'mr. reason'

In [ ]:

"""
00  adj.all	all adjective clusters
01	adj.pert	relational adjectives (pertainyms)
02	adv.all	all adverbs
03	noun.Tops	unique beginner for nouns
04	noun.act	nouns denoting acts or actions
05	noun.animal	nouns denoting animals
06	noun.artifact	nouns denoting man-made objects
07	noun.attribute	nouns denoting attributes of people and objects
08	noun.body	nouns denoting body parts
09	noun.cognition	nouns denoting cognitive processes and contents
10	noun.communication	nouns denoting communicative processes and contents
11	noun.event	nouns denoting natural events
12	noun.feeling	nouns denoting feelings and emotions
13	noun.food	nouns denoting foods and drinks
14	noun.group	nouns denoting groupings of people or objects
15	noun.location	nouns denoting spatial position
16	noun.motive	nouns denoting goals
17	noun.object	nouns denoting natural objects (not man-made)
18	noun.person	nouns denoting people
19	noun.phenomenon	nouns denoting natural phenomena
20	noun.plant	nouns denoting plants
21	noun.possession	nouns denoting possession and transfer of possession
22	noun.process	nouns denoting natural processes
23	noun.quantity	nouns denoting quantities and units of measure
24	noun.relation	nouns denoting relations between people or things or ideas
25	noun.shape	nouns denoting two and three dimensional shapes
26	noun.state	nouns denoting stable states of affairs
27	noun.substance	nouns denoting substances
28	noun.time	nouns denoting time and temporal relations
29	verb.body	verbs of grooming, dressing and bodily care
30	verb.change	verbs of size, temperature change, intensifying, etc.
31	verb.cognition	verbs of thinking, judging, analyzing, doubting
32	verb.communication	verbs of telling, asking, ordering, singing
33	verb.competition	verbs of fighting, athletic activities
34	verb.consumption	verbs of eating and drinking
35	verb.contact	verbs of touching, hitting, tying, digging
36	verb.creation	verbs of sewing, baking, painting, performing
37	verb.emotion	verbs of feeling
38	verb.motion	verbs of walking, flying, swimming
39	verb.perception	verbs of seeing, hearing, feeling
40	verb.possession	verbs of buying, selling, owning
41	verb.social	verbs of political and social activities and events
42	verb.stative	verbs of being, having, spatial relations
43	verb.weather	verbs of raining, snowing, thawing, thundering
44	adj.ppl   participial adjectives
"""

In [ ]:
def merge_duplicate_characters(rwn={'unique_id': '3470_captain_marvel_jr_bird_of_doom'}):
    df_entities = pd.read_csv('{}/{}.entities'.format(rwn['unique_id']), delimiter="\t")

    # get only personal pronoun elemnts
    df_entities = df_entities[(df_entities['prop']=='PROP') & (df_entities['cat']=='PER')].reset_index()
    from_list = list(df_entities['text'])
    to_list = list(df_entities['text'])
    print(from_list)
    matches = PolyFuzz("TF-IDF").match(from_list, to_list).get_matches()
    return matches

In [ ]:
df_entities = pd.read_csv('{}/{}.entities'.format('3470_captain_marvel_jr_bird_of_doom', '3470_captain_marvel_jr_bird_of_doom'), delimiter="\t")
print(df_entities.shape)
# get only personal pronoun elemnts
df_entities = df_entities[(df_entities['prop']=='PROP') & (df_entities['cat']=='PER')].reset_index()
print(df_entities.shape)
df_entities.head()

(1195, 6)
(48, 7)


,index,COREF,start_token,end_token,prop,cat,text
0,11,5,150,151,PROP,PER,ted tebow
1,19,6,246,246,PROP,PER,zard
2,26,7,319,322,PROP,PER,ernst g . osborne
3,57,8,579,579,PROP,PER,freddy
4,88,8,750,750,PROP,PER,freddy


In [ ]:
# get only personal pronoun elemnts
df_entities = df_entities[(df_entities['prop']=='PROP') & (df_entities['cat']=='PER')].reset_index(drop=True)
from_list = list(df_entities['text'].unique())
to_list = list(df_entities['text'].unique())
print(len(from_list))
from_list

33


['ted tebow',
 'zard',
 'ernst g . osborne',
 'freddy',
 'helen hitler',
 'vultur',
 'wluman',
 'timothy peabody',
 'dullah krushid',
 'durah krushid',
 'krushid',
 'freddy freeman',
 'mike',
 'alonzo',
 'modesty',
 'jonathan silverspoon',
 'capt . marvel',
 'mr . silverspoon',
 'sivana',
 'napoleon hopalong cassidy',
 'mr . peters',
 'lily clarke',
 'professor l . popnjay',
 'peter',
 'monk murdock',
 'capt . marvel jr .',
 'monk murray',
 'murdock',
 'kn marvel',
 'perfessor',
 'monk mur',
 'tom macan',
 'amy']

In [ ]:
matched_character_dict

{'capt . marvel': {'capt . marvel', 'kn marvel'},
 'dullah krushid': {'dullah krushid', 'krushid'},
 'durah krushid': {'durah krushid', 'krushid'},
 'freddy freeman': {'freddy', 'freddy freeman'},
 'jonathan silverspoon': {'jonathan silverspoon', 'mr . silverspoon'},
 'monk murdock': {'monk murdock', 'murdock'},
 'monk murray': {'monk mur', 'monk murray'},
 'mr . peters': {'mr . peters', 'peter'}}

In [ ]:
tfidf = TFIDF(n_gram_range=(2, 2), min_similarity=0, model_id="TF-IDF-Sklearn")
edit = EditDistance(n_jobs=-1)
matches = PolyFuzz('TF-IDF').match(from_list, to_list, top_n=3).get_matches()
matches

,From,To,Similarity,To_2,Similarity_2,To_3,Similarity_3
0,ted tebow,ted tebow,1.0,None,0.000,None,0.000
1,zard,zard,1.0,None,0.000,None,0.000
2,ernst g . osborne,ernst g . osborne,1.0,None,0.000,None,0.000
3,freddy,freddy,1.0,freddy freeman,0.724,None,0.000
4,helen hitler,helen hitler,1.0,None,0.000,None,0.000
5,vultur,vultur,1.0,None,0.000,None,0.000
6,wluman,wluman,1.0,freddy freeman,0.131,None,0.000
7,timothy peabody,timothy peabody,1.0,None,0.000,None,0.000
8,dullah krushid,dullah krushid,1.0,krushid,0.664,durah krushid,0.475
9,durah krushid,durah krushid,1.0,krushid,0.716,dullah krushid,0.475


In [ ]:
matches = PolyFuzz('TF-IDF').match(['krushid'], to_list, top_n=3).get_matches()
matches

,From,To,Similarity,To_2,Similarity_2,To_3,Similarity_3
0,krushid,krushid,1.0,durah krushid,0.702,dullah krushid,0.649


In [ ]:
filtered_matches = matches[(matches['Similarity_2'] > 0.5) & (matches['Similarity_2'] < 1.0)].reset_index(drop=True)
filtered_matches.head()

,From,To,Similarity,To_2,Similarity_2,To_3,Similarity_3
0,freddy,freddy,1.0,freddy freeman,0.724,None,0.000
1,dullah krushid,dullah krushid,1.0,krushid,0.664,durah krushid,0.475
2,durah krushid,durah krushid,1.0,krushid,0.716,dullah krushid,0.475
3,krushid,krushid,1.0,durah krushid,0.716,dullah krushid,0.664
4,freddy freeman,freddy freeman,1.0,freddy,0.724,wluman,0.131


In [ ]:
filtered_matches.head(20)

,From,To,Similarity,To_2,Similarity_2,To_3,Similarity_3
0,freddy,freddy,1.0,freddy freeman,0.724,None,0.000
1,dullah krushid,dullah krushid,1.0,krushid,0.664,durah krushid,0.475
2,durah krushid,durah krushid,1.0,krushid,0.716,dullah krushid,0.475
3,krushid,krushid,1.0,durah krushid,0.716,dullah krushid,0.664
4,freddy freeman,freddy freeman,1.0,freddy,0.724,wluman,0.131
5,jonathan silverspoon,jonathan silverspoon,1.0,mr . silverspoon,0.728,mr . peters,0.103
6,mr . silverspoon,mr . silverspoon,1.0,jonathan silverspoon,0.728,mr . peters,0.142
7,mr . peters,mr . peters,1.0,peter,0.886,mr . silverspoon,0.142
8,peter,peter,1.0,mr . peters,0.886,None,0.000
9,monk murdock,monk murdock,1.0,murdock,0.861,monk mur,0.608


In [ ]:
# from_lst = list(filtered_matches['From'])
# to_1_lst = list(filtered_matches['To'])
# to_2_lst = list(filtered_matches['To_2'])

In [ ]:
# matched_character_dict = {}

# for idx, (from_char, to_1_char, to_2_char) in enumerate(zip(from_lst, to_1_lst, to_2_lst)):
#   if from_char in matched_character_dict:
#       if to_2_char not in matched_character_dict[from_char]:
#           # add to the set using same from char
#           matched_character_dict[from_char].add(to_2_char)

#           # if the to char length is more then reverse key value pairs
#           if len(from_char) < len(to_2_char):
#               matched_character_dict[to_2_char] = matched_character_dict.pop(from_char)

#           # check 

#       else:
#           # if both keys are already present then do not dpo anything
#           pass


#   else:
#       if to_2_char in matched_character_dict:
#             # add the from character to set only if it is not presernt already
#             if not from_char in matched_character_dict[to_2_char]:
#                 matched_character_dict[to_2_char].add(from_char)

#                 # see if the key lengths are different and set char name with largest length as key
#                 # delet the old key and add from char as new key
#                 if len(from_char) > len(to_2_char):
#                     matched_character_dict[from_char] = matched_character_dict.pop(to_2_char)

#       else:
#           # if both to and from  char names must not be in dict
#           # select key as largest length char name
#           if len(from_char) > len(to_2_char):
#             char_key = from_char
#           else:
#             char_key = to_2_char

#           matched_character_dict[char_key] = {from_char, to_2_char}
